# Making Markov Chains of Various Types

First we'll load the full list of pickled sequences *(this may take a moment or two)*.

In [1]:
import pickle

In [6]:
with open('all_melodies.pkl', 'r') as f:
    all_melodies = pickle.load(f)
with open('all_rhythms.pkl', 'r') as g:
    all_rhythms = pickle.load(g)

In [7]:
assert len(all_melodies) == len(all_rhythms)

Next, we'll define a **Markov** object that will store Markov chain information for chains of various orders and types.

In [20]:
class Markov(object):
    """A container for holding Markov chains of various orders, 
    where the events in the given 'current state' may occur temporally 
    before and/or after the note to be examined as the 'next state'.
    """
    
    def __init__(self, before = 0, after = 0, mode = 0):
        """Initialize a Markov object with `before` notes before the
        space to be filled, `after` notes after the space to be filled,
        and with `mode` equal to 0, 1 or 2 according to whether the 'current
        state' contains notes only before, only after, or both before and after
        the note to be filled in the 'next state'.
        
        Inputs: 
        before - int - number of notes before next state
        after - int - number of notes after next state
        mode - int in range(3) - mode of the chain(see above)
        
        Outputs - Markov object
        """
        
        if mode == 0:
            assert before != 0 and after == 0
        elif mode == 1:
            assert before == 0 and after != 0
        else:
            assert before != 0 and after != 0

        self.before = before
        self.after = after
        self.mode = mode
        self.state_dict = {}
        
    def add_data(self, seq, result):
        """Add one (current state -> next state) instance to the dictionary.
        Store each instance as a tally, to be normalized later.
        
        Inputs:
        seq - if mode = 0, seq is a tuple of length before
              if mode = 1, seq is a tuple of length after
              if mode = 2, seq is a list of length two, with first element
                           a tuple of length before and with second element
                           a tuple of length after
        result - int - single event
        
        Outputs:
        None
        """
        
        if self.mode == 0:
            assert isinstance(seq, tuple) and len(seq) == self.before
        elif self.mode == 1:
            assert isinstance(seq, tuple) and len(seq) == self.after
        else:
            assert (isinstance(seq, list) and len(seq) == 2 and
                    isinstance(seq[0], tuple) and len(seq[0]) == self.before and
                    isinstance(seq[1], tuple) and len(seq[1]) == self.after)
            
        if seq not in self.state_dict:
            self.state_dict[seq] = {result: 1}
        elif result in self.state_dict[seq]:
            self.state_dict[seq][result] += 1
        else:
            self.state_dict[seq][result] = 1
            
    def normalize(self):
        """Convert the state_dict dictionary from counts to probabilities.
        
        Inputs: None
        
        Outputs: None
        """
        
        for seq in self.state_dict:
            sum = 0
            if len(self.state_dict[seq]) == 1:
                continue
            for result in self.state_dict[seq]:
                sum += self.state_dict[seq][result]
            for result in self.state_dict[seq]:
                self.state_dict[seq][result] /= float(sum)
        
            

Now to test!

In [34]:
markov0 = Markov(1, 0, 0)
melody0 = all_melodies[0]
for i in range(len(melody0) - 1):
    markov0.add_data((melody0[i],), melody0[i + 1])
markov0.normalize()

In [35]:
print markov0.state_dict

{(58,): {60: 1}, (65,): {58: 1}, (60,): {62: 1}, (72,): {73: 1}, (73,): {68: 0.2, 75: 0.2, 76: 0.2, 78: 0.2, 65: 0.2}, (68,): {73: 0.3333333333333333, 70: 0.3333333333333333, 71: 0.3333333333333333}, (75,): {68: 1}, (70,): {73: 0.5, 76: 0.25, 70: 0.25}, (76,): {68: 3}, (71,): {68: 0.5, 76: 0.5}, (78,): {70: 1}}


In [36]:
print melody0

[72, 73, 75, 68, 73, 76, 68, 73, 68, 71, 76, 68, 71, 68, 70, 76, 68, 70, 70, 73, 78, 70, 73, 65, 58, 60, 62]


Yay, this handles traditional (order 1) Markov models perfectly!

In [37]:
markov0 = Markov(1, 0, 0)
melody2 = all_melodies[2]
for i in range(len(melody2) - 1):
    markov0.add_data((melody2[i]), melody2[i + 1])
markov0.normalize()

AssertionError: 

It seems as though the chords (shown below as inner lists) give us a bit of an issue, though.

In [38]:
print melody2

[61, 59, 58, 54, 56, 60, 58, 60, 49, 61, 47, 46, 42, 44, 60, 51, 58, 60, 51, 58, 60, 61, 49, [49, 59], [42, 58], [56, 68], [58, 70], [72, 60], [75, 63], [67, 79], [80, 68], [66, 78], [80, 68], [66, 78], [80, 63], 65, 72, 74, 72, 71, 69, [66, 67], 63, 77, 65, 74, 70, 66, 68, [80, 75], 72, 82, 77, 58, 53, [58, 53], 82, 77, [58, 53], 53, 58, 82, 77, 53, 58, 53, 58, 77, 82, 53, 58, [58, 53], 65, 67, 69, 70, 71, 72, 68, 70, 72, 73, 74, 75, 68, 70, 72, 73, 75, 77, 68, 70, 72, 73, 75, [77, 62], 53, 60, 62, 53, 60, 62, 63, 51, [51, 61], [60, 44], [58, 70], [72, 60], [74, 62], [65, 77], [81, 69], [82, 70], [80, 68], [82, 70], [80, 68], [58, 66, 54, 70], [66, 58, 70, 54], [58, 70, 66, 54], [58, 70, 66, 54], 65, 77, 75, 63, 73, 61, 72, 60, 58, 70, 56, 68, [82, 70], [72, 84], [74, 86], [64, 76], [65, 77], [67, 79], [67, 79], [65, 77], [67, 79], [80, 68], [67, 79], [65, 77], [80, 68], [67, 79], [65, 77], [64, 76], [64, 76], [67, 79], [65, 77], [67, 79], [80, 68], [82, 70], [67, 79], [80, 68], [65, 